<a href="https://colab.research.google.com/github/JhonatanPeixoto/JhonatanPeixoto/blob/main/Projeto_Final(Allan%2C_Fabio%2C_Jhonatan%2C_Vitor)_Estudo_de_Caso_TripAdvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install selenium

!apt-get update
!apt install chromium-chromedriver

!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, 
            '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.l

In [83]:
# Importar biblioteca de regex
import re

# Importar biblioteca nltk para remoção de stop words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Biblioteca para remoção de caracteres especiais
from unicodedata import normalize
from string import punctuation

In [84]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [85]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [86]:
browser = webdriver.Chrome('chromedriver', options=chrome_options)

In [87]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303324-Goiania_State_of_Goias.html'

In [88]:
browser.get(url)

In [89]:
b = bs(browser.page_source)

In [90]:
rests = b.find('div', attrs={'data-test-target':'restaurants-list'})

In [91]:
class_alvo = rests.find('div').find('div')['class'][0]

In [92]:
df = pd.DataFrame(columns=['nome', 'avaliacao', 'descricao'])

In [93]:
linha = 0
for rest in rests.find_all('div', attrs={'class':class_alvo}):
    titulo = rest.find_all('a')[1].text
    avaliacao  = rest.find_all('a')[2].text
    desc   = rest.find_all('span')[14].text
    
    if re.search('\$', desc):
        desc   = rest.find_all('span')[12].text

    df.loc[linha] = [titulo, avaliacao, desc]
    linha+=1

###tratar coluna nome:remover numeração com regex, 
###tratar coluna avaliação: remover o nome "avaliações", converter para int
###tratar coluna descrição: caixa baixa, remover preposições, acentuações, etc


In [94]:
stop_words = stopwords.words('portuguese')

In [95]:
stop_words

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

In [96]:
def remover_sw(texto):
    words_token = word_tokenize(texto)
    
    # sentenca_limpa = []
    # for w in words_token:
    #     if not w.lower() in stop_words:
    #         sentenca_limpa.append(w)

    sentenca_limpa = [w for w in words_token if not w.lower() in stop_words]

    return ' '.join(sentenca_limpa)

In [97]:
def limpar_texto(texto):
    # Tratando caracteres especiais
    texto = normalize('NFKD', texto). encode('ASCII', 'ignore').decode('ASCII')
    # Removendo pontuações
    texto = re.sub(f'[{punctuation}]', '', texto)
    # Removendo número
    texto = re.sub('[0-9]', '', texto)
    # Colocar em caixa baixa
    texto = texto.lower()
    # Removendo stop words
    texto = remover_sw(texto)

    return texto

In [98]:
%%time
df['descricao'] = df['descricao'].map(limpar_texto)

CPU times: user 6.85 ms, sys: 44 µs, total: 6.89 ms
Wall time: 14.5 ms


In [116]:
new = df['nome'].str.split(".", n = 1, expand = True)

In [ ]:
new

In [118]:
df['nome'] = new[1]

In [119]:
df

,nome,avaliacao,descricao
0,Coco Bambu Flamboyant,4.983 avaliações,brasileira frutos mar
1,Taji Japanese Cuisine,828 avaliações,japonesa asiatica
2,El Argentino,334 avaliações,steakhouse argentina
3,Íz Restaurante,872 avaliações,francesa brasileira
4,Restaurante Popular,311 avaliações,brasileira
5,Ciao Ciao Cucina,178 avaliações,italiana bar
6,Carne de Sol 1008,2.341 avaliações,steakhouse brasileira
7,Madero,1.141 avaliações,americana steakhouse
8,Pizzaria Pitigliano,721 avaliações,italiana pizza
9,Outback Steakhouse - Shopping Flamboyant,1.807 avaliações,steakhouse bar


##Aplicação do sistema de recomendação

In [ ]:
df['descricao']

In [101]:
tipo_culinaria = df['descricao'].str.split(' ').astype(str)

In [ ]:
tipo_culinaria.head(30)

In [103]:
#importação de bibliotecas para o procedimento

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [104]:
# Criando uma instância do tfidf
vectorizer = TfidfVectorizer(analyzer='word',
                             ngram_range=(1,3))

# Definir os valores da nossa matriz de características
vetores = vectorizer.fit_transform(tipo_culinaria)

In [105]:
# Criar um dataframe com as características (features) e os pesos
# associados a estas
words_df = pd.DataFrame(vetores.toarray(),
                        columns=vectorizer.get_feature_names_out())

In [106]:
cosseno_tipo_culinaria = cosine_similarity(words_df, words_df)


In [107]:
df.shape


(30, 3)

In [ ]:
cosseno_tipo_culinaria[0][:]

In [ ]:
for i,v in enumerate(cosseno_tipo_culinaria[2]):
    if v > 0.4:
        print(i, v)

In [121]:
catalogo_restaurantes = pd.Series(df.index, index=df['nome'])

nome
 Coco Bambu Flamboyant                        0
 Taji Japanese Cuisine                        1
 El Argentino                                 2
 Íz Restaurante                               3
 Restaurante Popular                          4
 Ciao Ciao Cucina                             5
 Carne de Sol 1008                            6
 Madero                                       7
 Pizzaria Pitigliano                          8
 Outback Steakhouse - Shopping Flamboyant     9
 Glória Bar e Restaurante                    10
 Panela Mágica                               11
 Celsin & Cia                                12
 Lifebox Burger Gourmet                      13
 Madero Flamboyant Shopping                  14
 Botequim Mercatto                           15
 Chão Nativo                                 16
 Bahrem                                      17
 Casa São Paulo                              18
 Porto Cave                                  19
 Churrascaria Gramado              

In [126]:
import string
def recomendar_restaurante(nome_restaurante, mat_similaridades):
    # Obtendo o índice com base no título informado
    idx = catalogo_restaurantes[nome_restaurante] # 'barriga cheia'

    # Obter similaridades para o item selecionado
    sim = (enumerate(mat_similaridades[idx]))

    # Reordenar nossas similaridades em ordem decrescente de recomendação
    sim = sorted(sim, key=lambda x:x[1], reverse=True)

    # Filtrar os top 10
    sim = sim[0:11] 

    # Obter os índices de restaurantes recomendados
    indices_recomendados = [x[0] for x in sim]
    recomendados_sim     = [x[1] for x in sim]

    return pd.DataFrame({
        'Nome do restaurante':df['nome'].iloc[indices_recomendados].values,
        'Similaridade':recomendados_sim,
        'Avaliação:':df['avaliacao'].iloc[indices_recomendados].values,
        'Descricao:':df['descricao'].iloc[indices_recomendados].values,
    })

In [137]:
recomendar_restaurante(27, cosseno_tipo_culinaria)



,Nome do restaurante,Similaridade,Avaliação:,Descricao:
0,Coco Bambu Flamboyant,1.000000,4.983 avaliações,brasileira frutos mar
1,Bar do Peixe,1.000000,306 avaliações,brasileira frutos mar
2,Tucunaré na Chapa,1.000000,346 avaliações,brasileira frutos mar
3,Restaurante Popular,0.213533,311 avaliações,brasileira
4,Chão Nativo,0.213533,441 avaliações,brasileira
5,Glória Bar e Restaurante,0.084397,310 avaliações,brasileira bar
6,Celsin & Cia,0.084397,450 avaliações,brasileira bar
7,Bahrem,0.084397,420 avaliações,brasileira bar
8,Empório Saccaria,0.084397,777 avaliações,brasileira bar
9,Empório Piquiras,0.084397,993 avaliações,brasileira bar
